# AER - Access Entitlement Review Reporter

從 SharePoint 讀取審核人員的 Excel 回應，產生缺漏報告與合併報告。

## Cell 1: 設定與認證

In [ ]:
# === 設定與認證 (修改版：人肉登入模式) ===
import os
import logging
from datetime import datetime
from dotenv import load_dotenv
from msal import PublicClientApplication # <--- 改用 Public
import requests

load_dotenv()

# === Logging 設定 ===
os.makedirs("output", exist_ok=True)
log_file = f"output/aer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logger = logging.getLogger("aer")
logger.handlers.clear()  # 避免重複執行時累積 handler
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)

fh = logging.FileHandler(log_file)
fh.setFormatter(formatter)
logger.addHandler(fh)

os.makedirs("output", exist_ok=True)
log_file = f"output/aer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logger = logging.getLogger("aer")
logger.handlers.clear()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)
fh = logging.FileHandler(log_file)
fh.setFormatter(formatter)
logger.addHandler(fh)

# === Azure AD 設定 ===
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
# CLIENT_SECRET 不需要了！因為是用你的人來登入
# CLIENT_SECRET = os.getenv("AZURE_CLIENT_SECRET") 

# === SharePoint 設定 ===
SHAREPOINT_HOST = os.getenv("SHAREPOINT_HOST", "davidshih.sharepoint.com")
SITE_NAME = os.getenv("SITE_NAME", "aer")
APP_NAME = "2025 Entitlement Review/Q4/AVIDXCHANGE-TEST"
BASE_PATH = APP_NAME

# === Email 設定 ===
# 寄件者必須是你登入的那個帳號 (Delegated 限制)
SENDER_EMAIL = os.getenv("SENDER_EMAIL") 

# === 取得 Access Token (改用 Device Code Flow) ===
# 這裡定義你需要的所有 Delegated 權限
SCOPES = ["Sites.Read.All", "Mail.Send", "User.Read.All"]

app = PublicClientApplication(
    CLIENT_ID,
    authority=f"https://login.microsoftonline.com/{TENANT_ID}"
)

# 1. 發起登入請求
flow = app.initiate_device_flow(scopes=SCOPES)

if "user_code" not in flow:
    raise ValueError("無法建立 Device Flow，請檢查 Azure App 是否開啟 'Allow public client flows'")

# 2. 顯示登入指示 (這就是你要手動做的地方)
print("\n" + "="*60)
print(f"👉 請打開瀏覽器，前往: {flow['verification_uri']}")
print(f"👉 輸入代碼 (User Code): {flow['user_code']}")
print("="*60 + "\n")
logger.info("等待使用者登入中...")

# 3. 等待你完成登入
token_result = app.acquire_token_by_device_flow(flow)

if "access_token" not in token_result:
    error_msg = token_result.get('error_description', 'Unknown error')
    logger.error(f"認證失敗: {error_msg}")
    raise Exception(f"Azure AD 認證失敗: {error_msg}")

headers = {"Authorization": f"Bearer {token_result['access_token']}"}
logger.info(f"🎉 登入成功！使用者: {token_result.get('id_token_claims', {}).get('name')}")
logger.info(f"處理 app: {APP_NAME}")
logger.info(f"Log 檔案: {log_file}")

In [ ]:
# client secrete login mode

# === 設定與認證 (修改版：支援 Service Principal 機器人登入) ===
import os
import logging
from datetime import datetime
from dotenv import load_dotenv
from msal import PublicClientApplication, ConfidentialClientApplication # <--- 新增 Confidential
import requests

load_dotenv()

# === Logging 設定 ===
os.makedirs("output", exist_ok=True)
log_file = f"output/aer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logger = logging.getLogger("aer")
logger.handlers.clear()
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)

fh = logging.FileHandler(log_file)
fh.setFormatter(formatter)
logger.addHandler(fh)

# === Azure AD 設定 ===
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
CLIENT_SECRET = os.getenv("AZURE_CLIENT_SECRET") # <--- 這裡一定要有值才能用機器人登入

# === SharePoint 設定 ===
SHAREPOINT_HOST = os.getenv("SHAREPOINT_HOST", "davidshih.sharepoint.com")
SITE_NAME = os.getenv("SITE_NAME", "aer")
APP_NAME = "2025 Entitlement Review/Q4/AVIDXCHANGE-TEST"
BASE_PATH = APP_NAME

# === Email 設定 ===
SENDER_EMAIL = os.getenv("SENDER_EMAIL")

# === 取得 Access Token ===

if CLIENT_SECRET:
    # --- 模式 A: Service Principal (機器人登入 - 推薦！) ---
    # 優點: 不需要手動輸入代碼，通常可繞過裝置檢查
    logger.info("🤖 偵測到 Client Secret，使用 Service Principal (App-Only) 模式登入...")
    
    # App 權限通常用 .default
    SCOPES = ["https://graph.microsoft.com/.default"]
    
    app = ConfidentialClientApplication(
        CLIENT_ID,
        authority=f"https://login.microsoftonline.com/{TENANT_ID}",
        client_credential=CLIENT_SECRET
    )
    
    token_result = app.acquire_token_for_client(scopes=SCOPES)

else:
    # --- 模式 B: User Interactive (人肉登入) ---
    # 缺點: 會遇到 Error 530033 (裝置合規檢查)
    logger.info("👤 未偵測到 Client Secret，使用使用者互動 (Delegated) 模式登入...")
    
    SCOPES = ["Sites.Read.All", "Mail.Send", "User.Read.All"]
    
    app = PublicClientApplication(
        CLIENT_ID,
        authority=f"https://login.microsoftonline.com/{TENANT_ID}"
    )

    flow = app.initiate_device_flow(scopes=SCOPES)
    if "user_code" not in flow:
        raise ValueError("無法建立 Device Flow")

    print("\n" + "="*60)
    print(f"👉 請打開瀏覽器，前往: {flow['verification_uri']}")
    print(f"👉 輸入代碼 (User Code): {flow['user_code']}")
    print("="*60 + "\n")
    
    # 這裡已經幫你改成英文了，不會再 Crash
    logger.info("Waiting for user login...") 
    
    token_result = app.acquire_token_by_device_flow(flow)

# === 檢查結果 ===
if "access_token" not in token_result:
    error_msg = token_result.get('error_description', 'Unknown error')
    logger.error(f"認證失敗: {error_msg}")
    raise Exception(f"Azure AD 認證失敗: {error_msg}")

headers = {"Authorization": f"Bearer {token_result['access_token']}"}

# 取得名稱 (App 模式沒有名字，給個預設值)
user_name = token_result.get('id_token_claims', {}).get('name', 'Service Principal Bot')
logger.info(f"🎉 登入成功！使用者/App: {user_name}")
logger.info(f"處理 app: {APP_NAME}")

## Cell 2: SharePoint 函數

In [ ]:
# === Cell 2: SharePoint 函數 (Level 3: 路徑修復版) ===

def get_site_id(site_name: str) -> str:
    """取得 SharePoint site ID"""
    url = f"https://graph.microsoft.com/v1.0/sites/{SHAREPOINT_HOST}:/sites/{site_name}"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()["id"]


def list_folders(site_id: str, path: str) -> list[dict]:
    """列出資料夾下的所有子資料夾（自動判斷是否為根目錄）"""
    
    # 修正邏輯：如果路徑是空的，代表要抓根目錄 (Root)
    if not path or path.strip() == "":
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root/children"
    else:
        # 如果有路徑，才用冒號語法
        # 移除前後可能多餘的斜線
        clean_path = path.strip("/")
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/children"

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    
    return [
        {"name": item["name"], "webUrl": item.get("webUrl", "")}
        for item in resp.json().get("value", [])
        if item.get("folder")
    ]


def list_excel_files(site_id: str, folder_path: str) -> list[dict]:
    """列出資料夾下的 Excel 檔案"""
    # 同樣修正路徑邏輯
    clean_path = folder_path.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/children"
    
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    
    files = []
    for item in resp.json().get("value", []):
        if item["name"].endswith(".xlsx"):
            files.append({
                "id": item["id"],
                "name": item["name"],
                "lastModifiedDateTime": item.get("lastModifiedDateTime"),
                "webUrl": item.get("webUrl")
            })
    return sorted(files, key=lambda f: f.get("lastModifiedDateTime", ""), reverse=True)


def download_file(site_id: str, file_path: str) -> bytes:
    """下載檔案內容"""
    clean_path = file_path.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/content"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.content


def get_file_audit_log(site_id: str, file_id: str) -> str:
    """取得檔案版本歷史 (Audit Log)"""
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/items/{file_id}/versions"
    resp = requests.get(url, headers=headers)
    
    if resp.status_code != 200:
        return "無法取得版本紀錄"

    versions = resp.json().get("value", [])
    logs = []
    for v in versions:
        mod_time = v.get("lastModifiedDateTime", "")[:19].replace("T", " ")
        user_info = v.get("lastModifiedBy", {}).get("user", {})
        app_info = v.get("lastModifiedBy", {}).get("application", {})
        actor = user_info.get("displayName") or app_info.get("displayName") or "Unknown System"
        ver_num = v.get("id", "")
        logs.append(f"{mod_time} - {actor} (v{ver_num})")
        
    return "\n".join(logs)

# 重新測試連線
try:
    site_id = get_site_id(SITE_NAME)
    logger.info(f"SharePoint 連線成功，Site ID: {site_id[:20]}...")
    logger.info("函數庫已更新 (路徑修正版)")
except Exception as e:
    logger.error(f"SharePoint 連線失敗: {e}")
    raise

In [ ]:
# === Cell 2.5: 互動式資料夾選擇器 (Folder Browser 修正版) ===
import ipywidgets as widgets
from IPython.display import display, clear_output

# 🔥 關鍵修正：起始路徑設為空字串，代表直接從 drive/root 開始
current_path = "" 
selected_target_path = None

def create_folder_browser():
    if 'list_folders' not in globals() or 'site_id' not in globals():
        print("⚠️ 請先執行 Cell 2 以載入 SharePoint 函式！")
        return

    # --- UI 元件 ---
    path_label = widgets.HTML(value=f"<b>目前路徑:</b> Root (Shared Documents)")
    
    folder_dropdown = widgets.Dropdown(
        options=[],
        description='子資料夾:',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )
    
    btn_enter = widgets.Button(description='📂 進入', button_style='info', icon='folder-open')
    btn_back = widgets.Button(description='⬅️ 返回', button_style='', icon='arrow-left')
    btn_select = widgets.Button(description='✅ 設定為目標資料夾', button_style='success', icon='check')
    
    msg_output = widgets.Output()

    # --- 功能邏輯 ---
    def load_folders(path):
        try:
            items = list_folders(site_id, path)
            options = [item['name'] for item in items if item['name'] != "Forms"]
            folder_dropdown.options = sorted(options)
            
            if not options:
                folder_dropdown.disabled = True
                folder_dropdown.placeholder = "無子資料夾"
            else:
                folder_dropdown.disabled = False
                folder_dropdown.value = options[0] if options else None
                
        except Exception as e:
            with msg_output:
                print(f"❌ 讀取失敗: {e}")

    def on_enter_click(b):
        global current_path
        if folder_dropdown.value:
            # 修正路徑串接邏輯
            if current_path:
                current_path = f"{current_path}/{folder_dropdown.value}"
            else:
                current_path = folder_dropdown.value
                
            path_label.value = f"<b>目前路徑:</b> {current_path}"
            load_folders(current_path)

    def on_back_click(b):
        global current_path
        if "/" in current_path:
            current_path = current_path.rsplit("/", 1)[0]
        else:
            current_path = "" # 回到 Root
            
        path_label.value = f"<b>目前路徑:</b> {current_path or 'Root (Shared Documents)'}"
        load_folders(current_path)

    def on_select_click(b):
        global BASE_PATH
        BASE_PATH = current_path
        with msg_output:
            clear_output()
            print(f"🎯 已鎖定目標路徑: {BASE_PATH}")
            print("👉 請繼續執行 Cell 3 和 Cell 4。")

    btn_enter.on_click(on_enter_click)
    btn_back.on_click(on_back_click)
    btn_select.on_click(on_select_click)

    load_folders(current_path)

    ui = widgets.VBox([
        widgets.HTML("<h3>📂 SharePoint 資料夾瀏覽器</h3>"),
        path_label,
        widgets.HBox([btn_back, folder_dropdown, btn_enter]),
        widgets.HTML("<br>"),
        btn_select,
        msg_output
    ])
    display(ui)

create_folder_browser()

## Cell 3: Excel 讀取

In [ ]:
# === Excel 讀取（處理 AutoFilter visible rows）===
from openpyxl import load_workbook
from io import BytesIO

# 欄位名稱設定
COL_REVIEWER = "Reviewer"
COL_RESPONSE = "Reviewer's Response"
COL_DETAILS = "Details of Access change"


def read_visible_rows(excel_bytes: bytes, reviewer_name: str, file_name: str, folder_url: str) -> list[dict]:
    """
    讀取 Excel 的 visible rows（考慮 AutoFilter）
    
    Args:
        excel_bytes: Excel 檔案的 bytes
        reviewer_name: 審核人名稱（用於 fallback 過濾）
        file_name: 檔案名稱（追蹤用）
        folder_url: SharePoint 資料夾 URL（可點擊）
    
    Returns:
        list of ReviewerResponse dict
    """
    wb = load_workbook(BytesIO(excel_bytes))
    ws = wb.active
    
    # 找出欄位索引（假設第一行是標題）
    header_row = [cell.value for cell in ws[1]]
    col_map = {str(name).strip(): idx for idx, name in enumerate(header_row) if name}
    
    # 檢查必要欄位
    reviewer_col = col_map.get(COL_REVIEWER)
    response_col = col_map.get(COL_RESPONSE)
    details_col = col_map.get(COL_DETAILS)
    
    if reviewer_col is None or response_col is None:
        raise ValueError(f"找不到必要欄位。現有欄位: {list(col_map.keys())}")
    
    # 檢查是否有 hidden rows（表示有 AutoFilter）
    has_filter = any(
        ws.row_dimensions[i].hidden 
        for i in range(2, ws.max_row + 1) 
        if i in ws.row_dimensions
    )
    
    results = []
    for row_idx in range(2, ws.max_row + 1):
        # 如果有 filter，跳過被隱藏的行
        if has_filter and ws.row_dimensions.get(row_idx) and ws.row_dimensions[row_idx].hidden:
            continue
        
        row = [cell.value for cell in ws[row_idx]]
        
        # 取得欄位值
        reviewer = row[reviewer_col] if reviewer_col < len(row) else None
        response = row[response_col] if response_col < len(row) else None
        details = row[details_col] if details_col is not None and details_col < len(row) else None
        
        # 如果沒有 filter，用欄位值過濾
        if not has_filter:
            if reviewer is None or str(reviewer).strip().lower() != reviewer_name.lower():
                continue
        
        # 判斷是否缺漏
        is_missing = response is None or str(response).strip() == ""
        
        results.append({
            "reviewer": reviewer_name,
            "response": response,
            "details": details,
            "is_missing": is_missing,
            "row_number": row_idx,
            "file_name": file_name,
            "folder_url": folder_url
        })
    
    wb.close()
    return results


logger.info("Excel 讀取函數已載入")

## Cell 4: 資料收集

In [ ]:
# === Cell 4: 資料收集 (智慧偵測版：支援 App 層級與 User 層級) ===
import pandas as pd

# 確保 BASE_PATH 是正確的相對路徑
logger.info(f"目前設定路徑: {BASE_PATH}")
initial_folders = list_folders(site_id, BASE_PATH)

# === 智慧判斷邏輯 ===
# 判斷我們現在是在 "Q4" (看到一堆 App) 還是在 "App" (看到一堆 User)
# 方法：檢查第一個資料夾，如果裡面包含 "審核 Excel"，代表我們選對了。
# 如果裡面包含的是 "資料夾"，代表我們選太高層了。

target_app_folders = [] # 格式: (App名稱, 完整路徑)

# 先偷看第一個資料夾的內容
if initial_folders:
    first_folder_name = initial_folders[0]["name"]
    first_folder_path = f"{BASE_PATH}/{first_folder_name}"
    
    # 看看裡面有沒有 Excel
    files_inside = list_excel_files(site_id, first_folder_path)
    is_user_folder = any(first_folder_name.lower() in f["name"].lower() for f in files_inside)
    
    if is_user_folder:
        # 狀況 A: 我們選到了 App 層級 (裡面直接是 User) -> 舊邏輯
        logger.info("✅ 偵測到直接路徑 (Single App Mode)")
        target_app_folders.append((APP_NAME, BASE_PATH))
    else:
        # 狀況 B: 我們選到了 Q4 層級 (裡面是 App 資料夾) -> 批次邏輯
        logger.info("🔄 偵測到上層路徑 (Batch App Mode) - 將掃描所有子 App")
        for f in initial_folders:
            # 把 Q4 底下的每個 App 都加入排程
            app_sub_path = f"{BASE_PATH}/{f['name']}"
            target_app_folders.append((f['name'], app_sub_path))
else:
    logger.warning("❌ 目標路徑下是空的！")

# === 開始跑迴圈 ===
all_responses = []
errors = []

for current_app_name, current_path in target_app_folders:
    logger.info(f"🚀 開始處理 App: {current_app_name} (路徑: {current_path})")
    
    # 抓取該 App 下的所有審核人
    try:
        reviewers = list_folders(site_id, current_path)
    except Exception as e:
        logger.error(f"  無法讀取 App 資料夾: {e}")
        continue
        
    for folder in reviewers:
        reviewer_name = folder["name"]
        folder_url = folder["webUrl"]
        folder_path = f"{current_path}/{reviewer_name}"
        
        try:
            # 列出 Excel 檔案
            excel_files = list_excel_files(site_id, folder_path)
            
            # 找到包含審核人名字的 Excel 檔案
            target_files = [f for f in excel_files if reviewer_name.lower() in f["name"].lower()]
            
            if not target_files:
                # 靜默跳過系統資料夾或不相關的
                if reviewer_name not in ["Forms", "_private"]:
                    logger.warning(f"  [跳過] {reviewer_name}: 找不到對應 Excel")
                continue
            
            # 讀取最新的檔案
            target_file = target_files[0]
            file_name = target_file["name"]
            file_id = target_file["id"]
            file_download_path = f"{folder_path}/{file_name}"
            
            logger.info(f"  📖 讀取: {current_app_name} -> {reviewer_name}")
            
            # 抓取 Audit Log
            audit_log_str = get_file_audit_log(site_id, file_id)
            
            # 下載並解析
            excel_bytes = download_file(site_id, file_download_path)
            rows = read_visible_rows(excel_bytes, reviewer_name, file_name, folder_url)
            
            # 補上 App Name 欄位 (這樣合併報告才知道是哪個 App)
            for row in rows:
                row["App_Name"] = current_app_name
                row["Audit_History"] = audit_log_str
                row["Last_Modified"] = target_file["lastModifiedDateTime"]
            
            all_responses.extend(rows)
            
        except Exception as e:
            logger.error(f"  ❌ 失敗 {reviewer_name}: {e}")
            errors.append({"reviewer": reviewer_name, "error": str(e), "folder_url": folder_url})

# 轉成 DataFrame
df = pd.DataFrame(all_responses)
logger.info(f"\n=== 收集完成 ===")
logger.info(f"總計: {len(all_responses)} 筆記錄")
logger.info(f"缺漏: {df['is_missing'].sum() if len(df) > 0 else 0} 筆")

## Cell 5: 互動式狀態表格

In [9]:
# === Cell 5: 互動式狀態表格 (含 Excel 輸出與 AutoFit) ===
from IPython.display import display, HTML
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from datetime import datetime

# 取得時間戳記
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
safe_app_name = APP_NAME.replace("/", "_").replace("\\", "_")

def format_summary_excel(file_path: str):
    """
    [美化函數] 專為摘要表設計的 AutoFit
    """
    try:
        wb = load_workbook(file_path)
        ws = wb.active
        
        # 遍歷每一欄
        for column_cells in ws.columns:
            header_cell = column_cells[0]
            col_letter = get_column_letter(header_cell.column)
            
            # 計算最大寬度
            max_length = 0
            for cell in column_cells:
                try:
                    if cell.value:
                        cell_len = len(str(cell.value))
                        # 如果是連結網址，不要撐太大，設個上限
                        if "http" in str(cell.value) and cell_len > 50:
                            cell_len = 50
                        if cell_len > max_length:
                            max_length = cell_len
                except:
                    pass
            
            # 設定寬度 (加一點緩衝)
            adjusted_width = (max_length + 2) * 1.1
            if adjusted_width > 60: adjusted_width = 60 # 上限
            
            ws.column_dimensions[col_letter].width = adjusted_width
            
            # 統一靠上對齊
            for cell in column_cells:
                cell.alignment = Alignment(vertical='top')

        wb.save(file_path)
        # logger.info(f"✨ 摘要表已美化: {file_path}") # 註解掉避免洗版
        
    except Exception as e:
        logger.warning(f"摘要表美化失敗: {e}")

def create_status_table(df: pd.DataFrame) -> pd.DataFrame:
    """建立互動式狀態表格 (含上層目錄顯示 + Excel 匯出)"""
    
    if len(df) == 0:
        display(HTML("<p style='color: red;'>沒有資料可顯示</p>"))
        return pd.DataFrame()
    
    # --- 1. 抓取層級名稱 (例如 "Q4") ---
    try:
        parent_folder = BASE_PATH.strip("/").split("/")[-1]
    except:
        parent_folder = "Root"
        
    if "App_Name" not in df.columns:
        df["App_Name"] = APP_NAME

    # --- 2. 組合顯示名稱 ---
    df["Display_App_Name"] = df["App_Name"].apply(
        lambda x: f"{parent_folder} / {x.split('/')[-1] if '/' in x else x}"
    )

    # --- 3. 統計與分組 ---
    summary = df.groupby(["Display_App_Name", "reviewer", "folder_url"]).agg(
        total=("reviewer", "count"),
        missing=("is_missing", "sum")
    ).reset_index()
    
    summary["status"] = summary.apply(
        lambda r: "🔴 未完成" if r["missing"] > 0 else "🟢 完成", axis=1
    )
    summary["completion"] = ((summary["total"] - summary["missing"]) / summary["total"] * 100).round(1)
    
    # --- 4. 排序邏輯 ---
    summary_sorted = summary.sort_values(
        ["missing", "Display_App_Name", "reviewer"], 
        ascending=[False, True, True]
    )
    
    # --- 5. 匯出 Excel 並美化 (新增功能) ---
    summary_file = f"output/summary_status_all-apps_{timestamp}.xlsx"
    
    # 整理要輸出的欄位 (把 URL 放最後)
    export_df = summary_sorted[[
        "Display_App_Name", "reviewer", "status", "completion", "missing", "total", "folder_url"
    ]].copy()
    
    export_df.to_excel(summary_file, index=False)
    format_summary_excel(summary_file) # <--- 呼叫美化魔法
    
    # --- 6. 建立 HTML ---
    def make_row(r):
        bg_color = "#ffcccc" if r["missing"] > 0 else "#ccffcc"
        return f"""
        <tr style="background-color: {bg_color};">
            <td style="font-weight:bold;">{r['Display_App_Name']}</td>
            <td><a href="{r['folder_url']}" target="_blank">{r['reviewer']}</a></td>
            <td>{r['status']}</td>
            <td>{r['completion']}%</td>
            <td>{int(r['missing'])} / {int(r['total'])}</td>
        </tr>
        """
    
    html = f"""
    <style>
        .status-table {{ border-collapse: collapse; width: 100%; font-family: sans-serif; }}
        .status-table th, .status-table td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
        .status-table th {{ background-color: #4CAF50; color: white; }}
        .status-table a {{ color: #0066cc; text-decoration: none; font-weight: bold; }}
        .status-table a:hover {{ text-decoration: underline; }}
    </style>
    <h3>📊 審核狀態總覽</h3>
    <p>
        📥 已匯出摘要 Excel (AutoFit): <code>{summary_file}</code>
    </p>
    <table class="status-table">
        <tr>
            <th>App 名稱 (層級)</th>
            <th>審核人</th>
            <th>狀態</th>
            <th>完成率</th>
            <th>缺漏/總數</th>
        </tr>
        {''.join(summary_sorted.apply(make_row, axis=1))}
    </table>
    <p>
        <b>總計:</b> {len(summary)} 位審核人, 
        <b>{int(summary['missing'].sum())}</b> 筆缺漏
    </p>
    """
    
    display(HTML(html))
    return summary

# 執行
summary_df = create_status_table(df)

App 名稱 (層級),審核人,狀態,完成率,缺漏/總數
Q4 / AVIDXCHANGE-TEST,Joe Petti,🔴 未完成,50.0%,2 / 4
Q4 / Visa,Charlie Brown,🔴 未完成,60.0%,2 / 5
Q4 / Visa,Joe Petti,🔴 未完成,50.0%,2 / 4
Q4 / AVIDXCHANGE-TEST,Jane Smith,🔴 未完成,80.0%,1 / 5
Q4 / AVIDXCHANGE-TEST,Bob O'Brien,🟢 完成,100.0%,0 / 3
Q4 / Visa,Alice Wonderland,🟢 完成,100.0%,0 / 5


## Cell 6: 發送提醒郵件

In [10]:
# === Cell 6: 互動式郵件控制塔台 (Table UI 版) ===
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
from urllib.parse import quote
import time

# === 1. API 函式庫 ===

def get_user_email(name: str) -> str:
    """嘗試從 Graph API 搜尋 User Email"""
    try:
        # 先清除名字中可能導致搜尋失敗的雜訊
        clean_name = name.split("(")[0].strip()
        encoded_name = quote(clean_name)
        
        # 策略 A: 精確搜尋 DisplayName
        url = f"https://graph.microsoft.com/v1.0/users?$filter=displayName eq '{encoded_name}'&$select=userPrincipalName,mail"
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200 and resp.json().get("value"):
            user = resp.json()["value"][0]
            return user.get("mail") or user.get("userPrincipalName")
        
        # 策略 B: 模糊搜尋 (startswith)
        url = f"https://graph.microsoft.com/v1.0/users?$filter=startswith(displayName, '{encoded_name}')&$select=userPrincipalName,mail"
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200 and resp.json().get("value"):
            user = resp.json()["value"][0]
            return user.get("mail") or user.get("userPrincipalName")
            
    except Exception as e:
        pass
    return "" # 找不到回傳空字串

def send_mail_api(sender: str, to: str, subject: str, body: str) -> tuple[bool, str]:
    """呼叫 Graph API 寄信 (支援指定 Sender)"""
    try:
        # 注意: 如果 sender 不是當前登入者，需要有 Shared Mailbox 權限
        url = f"https://graph.microsoft.com/v1.0/users/{sender}/sendMail"
        
        data = {
            "message": {
                "subject": subject,
                "body": {"contentType": "HTML", "content": body},
                "toRecipients": [{"emailAddress": {"address": to}}]
            },
            "saveToSentItems": "true"
        }
        
        resp = requests.post(url, headers={**headers, "Content-Type": "application/json"}, json=data)
        
        if resp.status_code == 202:
            return True, "已發送"
        else:
            return False, f"失敗 ({resp.status_code}): {resp.text}"
    except Exception as e:
        return False, str(e)

# === 2. UI 建構 ===

# 檢查是否有資料
if 'summary_df' not in dir() or len(summary_df) == 0:
    display(widgets.HTML("<h3 style='color:red'>⚠️ 請先執行前面的 Cell 產生 summary_df</h3>"))
else:
    # 篩選出未完成的人
    targets = summary_df[summary_df['missing'] > 0].copy()
    
    if len(targets) == 0:
        display(widgets.HTML("<h3 style='color:green'>🎉 恭喜！所有審核都已完成，無需寄信。</h3>"))
    else:
        # --- 全域設定區 ---
        
        # 預設寄件者 (從環境變數或預設值)
        default_sender = SENDER_EMAIL if 'SENDER_EMAIL' in globals() and SENDER_EMAIL else "me"
        
        style = {'description_width': 'initial'}
        
        global_subject = widgets.Text(
            value="[Action Required] 2025 Access Review Pending",
            description="<b>郵件主旨:</b>",
            layout=widgets.Layout(width='95%'),
            style=style
        )
        
        default_template = """<p>Hi {name},</p>
<p>溫馨提醒：您在 <b>{app}</b> 還有 <b>{missing}</b> 筆權限審核尚未完成。</p>
<p>請點擊連結進行作業：<a href="{link}">開啟資料夾</a></p>
<p>謝謝</p>"""

        global_body_template = widgets.Textarea(
            value=default_template,
            description="<b>內容模板:</b>",
            placeholder="使用 {name}, {app}, {missing}, {link} 作為變數",
            layout=widgets.Layout(width='95%', height='100px'),
            style=style
        )
        
        btn_apply_template = widgets.Button(
            description="套用模板到下方所有列",
            button_style='info',
            icon='arrow-down'
        )
        
        # --- 表格標題 ---
        header_html = widgets.HTML(
            """
            <div style="display: flex; font-weight: bold; background-color: #f0f0f0; padding: 8px; border-bottom: 2px solid #ccc;">
                <div style="width: 30px;">選</div>
                <div style="width: 150px;">App / Reviewer</div>
                <div style="width: 200px;">收件人 (To)</div>
                <div style="width: 200px;">寄件人 (From)</div>
                <div style="flex: 1;">郵件內容 (可編輯)</div>
                <div style="width: 80px;">操作</div>
            </div>
            """
        )

        # --- 產生每一列 (Rows) ---
        row_widgets_list = [] # 存 UI 物件
        row_data_map = {}     # 存資料參照
        
        print("⏳ 正在從 Azure AD 搜尋 Email，請稍候...")
        
        for idx, row in targets.iterrows():
            r_name = row['reviewer']
            r_app = row['Display_App_Name']
            r_miss = int(row['missing'])
            r_link = row['folder_url']
            
            # 1. 自動找 Email
            found_email = get_user_email(r_name)
            
            # 2. 預設內容
            initial_body = default_template.format(
                name=r_name, app=r_app, missing=r_miss, link=r_link
            )
            
            # --- UI 元件 ---
            w_check = widgets.Checkbox(value=True, indent=False, layout=widgets.Layout(width='30px'))
            
            w_info = widgets.HTML(
                f"<div style='font-size:12px; line-height:1.2'><b>{r_app}</b><br>{r_name}<br><span style='color:red'>缺 {r_miss} 筆</span></div>",
                layout=widgets.Layout(width='150px')
            )
            
            w_to = widgets.Text(
                value=found_email, 
                placeholder="輸入 Email",
                layout=widgets.Layout(width='190px')
            )
            
            w_from = widgets.Text(
                value=default_sender,
                placeholder="me 或 user@com...",
                layout=widgets.Layout(width='190px')
            )
            
            w_body = widgets.Textarea(
                value=initial_body,
                layout=widgets.Layout(width='98%', height='60px')
            )
            
            w_btn_send = widgets.Button(
                description="🚀", 
                layout=widgets.Layout(width='40px'),
                button_style='warning',
                tooltip='單獨發送此封'
            )
            
            w_status = widgets.Label(value="", layout=widgets.Layout(width='80px'))
            
            # 組合一列
            row_box = widgets.HBox(
                [w_check, w_info, w_to, w_from, w_body, widgets.VBox([w_btn_send, w_status])],
                layout=widgets.Layout(border='1px solid #eee', padding='5px', align_items='center')
            )
            
            # 存起來方便後續操作
            row_key = f"{r_app}_{r_name}"
            row_data_map[row_key] = {
                "check": w_check, "to": w_to, "from": w_from, "body": w_body, 
                "status": w_status, "btn": w_btn_send,
                "data": {"name": r_name, "app": r_app, "missing": r_miss, "link": r_link}
            }
            row_widgets_list.append(row_box)
            
            # --- 單獨發送邏輯 ---
            def create_send_handler(key):
                def on_click(b):
                    widgets_row = row_data_map[key]
                    to_addr = widgets_row['to'].value.strip()
                    from_addr = widgets_row['from'].value.strip()
                    body_txt = widgets_row['body'].value
                    subject_txt = global_subject.value
                    
                    if not to_addr:
                        widgets_row['status'].value = "❌ 無 Email"
                        return

                    widgets_row['btn'].disabled = True
                    widgets_row['status'].value = "⏳ 發送中..."
                    
                    success, msg = send_mail_api(from_addr, to_addr, subject_txt, body_txt)
                    
                    if success:
                        widgets_row['status'].value = "✅ 已寄出"
                        widgets_row['check'].value = False # 寄完自動取消勾選
                        widgets_row['btn'].button_style = 'success'
                    else:
                        widgets_row['status'].value = "❌ 失敗"
                        print(f"[{key}] 發送失敗: {msg}")
                        widgets_row['btn'].disabled = False
                return on_click

            w_btn_send.on_click(create_send_handler(row_key))

        clear_output() # 清除 "正在搜尋..." 的訊息

        # --- 批量操作邏輯 ---
        
        # 1. 套用模板按鈕
        def on_apply_template(b):
            tpl = global_body_template.value
            for key, w_row in row_data_map.items():
                if w_row['check'].value: # 只套用在有勾選的
                    d = w_row['data']
                    new_body = tpl.format(name=d['name'], app=d['app'], missing=d['missing'], link=d['link'])
                    w_row['body'].value = new_body
        btn_apply_template.on_click(on_apply_template)
        
        # 2. 全部發送按鈕
        btn_send_all = widgets.Button(
            description="⚡ 發送已選取的郵件 (Bulk Send)",
            button_style='danger',
            layout=widgets.Layout(width='100%'),
            icon='paper-plane'
        )
        
        log_output = widgets.Output()
        
        def on_send_all(b):
            with log_output:
                clear_output()
                count = 0
                for key, w_row in row_data_map.items():
                    if w_row['check'].value:
                        # 觸發該列的按鈕點擊事件
                        w_row['btn'].click()
                        count += 1
                        time.sleep(0.5) # 稍微間隔避免 API Rate Limit
                print(f"已執行 {count} 封郵件的發送程序。")

        btn_send_all.on_click(on_send_all)

        # --- 最終組裝 ---
        ui = widgets.VBox([
            widgets.HTML("<h3>📧 郵件發送控制台</h3>"),
            global_subject,
            widgets.HBox([global_body_template, widgets.VBox([widgets.Label(" "), btn_apply_template])]),
            widgets.HTML("<hr>"),
            header_html,
            widgets.VBox(row_widgets_list, layout=widgets.Layout(max_height='500px', overflow_y='auto')), # 加上卷軸
            widgets.HTML("<hr>"),
            btn_send_all,
            log_output
        ])
        
        display(ui)

## Cell 7: 產生報告

In [ ]:
# === Cell 7: 完整結案報告 (含成功報表 & 錯誤報表) ===
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime

# 取得時間戳記
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# 處理檔名非法字元
safe_app_name = APP_NAME.replace("/", "_").replace("\\", "_")

def format_excel_layout(file_path: str):
    """
    [美化函數] 智慧型欄寬調整：
    1. 長文欄位 -> 固定寬度 + 自動換行
    2. 短文欄位 -> 自動偵測內容長度 (AutoFit)
    """
    try:
        wb = load_workbook(file_path)
        ws = wb.active
        
        # 定義需要「固定寬度 + 換行」的長欄位名稱
        long_text_cols = ["Audit_History", "Details of Access change", "details", "Details", "response", "error"]
        
        for column_cells in ws.columns:
            header_cell = column_cells[0]
            header_val = str(header_cell.value).strip() if header_cell.value else ""
            col_letter = get_column_letter(header_cell.column)
            
            # === 策略 A: 長文欄位 (固定寬度 + 換行) ===
            if header_val in long_text_cols:
                ws.column_dimensions[col_letter].width = 50
                for cell in column_cells:
                    cell.alignment = Alignment(wrap_text=True, vertical='top')
            
            # === 策略 B: 一般欄位 (AutoFit) ===
            else:
                max_length = 0
                for cell in column_cells:
                    try:
                        if cell.value:
                            cell_len = len(str(cell.value))
                            if cell_len > max_length: max_length = cell_len
                    except: pass
                
                adjusted_width = (max_length + 2) * 1.1
                if adjusted_width > 60: adjusted_width = 60
                ws.column_dimensions[col_letter].width = adjusted_width
                
                for cell in column_cells:
                    current_align = cell.alignment
                    cell.alignment = Alignment(vertical='top', horizontal=current_align.horizontal)

        wb.save(file_path)
        logger.info(f"✨ 已美化 Excel: {file_path}")
        
    except Exception as e:
        logger.warning(f"Excel 格式美化失敗: {e}")

# ==========================================
# Part 1: 成功資料報告 (Success Reports)
# ==========================================
print("--- 正在產生成功報告 ---")
report_html = ""

if len(df) > 0:
    export_cols = [
        "App_Name", "reviewer", "response", "details", "is_missing", 
        "Audit_History", "Last_Modified", 
        "row_number", "file_name", "folder_url"
    ]
    
    # 1. 缺漏報告
    missing_df = df[df["is_missing"]].copy()
    missing_file_xlsx = f"output/missing_responses_{safe_app_name}_{timestamp}.xlsx"
    cols_to_save = [c for c in export_cols if c in missing_df.columns]
    missing_df[cols_to_save].to_excel(missing_file_xlsx, index=False)
    format_excel_layout(missing_file_xlsx)
    
    # 2. 合併報告
    consolidated_file_xlsx = f"output/consolidated_report_{safe_app_name}_{timestamp}.xlsx"
    cols_to_save = [c for c in export_cols if c in df.columns]
    df[cols_to_save].to_excel(consolidated_file_xlsx, index=False)
    format_excel_layout(consolidated_file_xlsx)
    
    report_html += f"""
    <h3>✅ 成功資料報告</h3>
    <ul>
        <li>缺漏報告: <code>{missing_file_xlsx}</code></li>
        <li>合併報告: <code>{consolidated_file_xlsx}</code></li>
    </ul>
    """
else:
    report_html += "<p style='color: orange;'>⚠️ 沒有成功讀取到的資料，無法產生合併報告。</p>"

# ==========================================
# Part 2: 錯誤報告 (Error Reports) - 原 Cell 8
# ==========================================
print("--- 正在檢查錯誤 ---")

if errors:
    # 轉成 DataFrame
    error_df = pd.DataFrame(errors)
    
    # 改存成 XLSX 以支援美化
    error_file_xlsx = f"output/errors_{safe_app_name}_{timestamp}.xlsx"
    error_df.to_excel(error_file_xlsx, index=False)
    format_excel_layout(error_file_xlsx)
    
    logger.warning(f"有 {len(errors)} 個審核人處理失敗")
    
    # 產生錯誤表格 HTML
    def make_error_row(r):
        return f"""
        <tr style="background-color: #fff3cd;">
            <td><a href="{r['folder_url']}" target="_blank">{r['reviewer']}</a></td>
            <td>{r['error']}</td>
        </tr>
        """
    
    report_html += f"""
    <hr>
    <h3 style="color: #d9534f;">⚠️ 處理錯誤報告 ({len(errors)} 筆)</h3>
    <p>錯誤清單已儲存: <code>{error_file_xlsx}</code></p>
    <table class="status-table">
        <tr>
            <th>審核人</th>
            <th>錯誤訊息</th>
        </tr>
        {''.join(error_df.apply(make_error_row, axis=1))}
    </table>
    """
else:
    report_html += "<hr><p style='color: green;'>🎉 太棒了！沒有任何讀取錯誤。</p>"

# 顯示最終結果
display(HTML(report_html))